# CNNbasic Assignment#2

# AlexNet 구현

모델 구현 후 summary로 전체 모델 구조 출력과 주석으로 간단한 설명을 달아주시면 됩니다.

프레임워크는 자유이고, 기본 tensforflow와 pytorch tutorial 사이트를 아래에 첨부해 드립니다.

이 외 각 프레임워크 별 summary 등 추가적인 사용 방법은 구글링으로 찾아주세요!-!

- Tensorflow Tutorial: https://www.tensorflow.org/tutorials?hl=ko

- Pytorch Tutorial: https://tutorials.pytorch.kr/

![image-2.png](attachment:image-2.png)

- 논문 컨셉

    논문제목은 “ImageNet Classification with Deep Convolution Neural Networks”

    이미지 관련 모델로서, 2012년 ILSVRC에서 큰 격차로 1등을 기록. AlexNet은 Alex Krizhevsky라는 1저자가 논문을 써서 이름이 알렉스


- 모델 특징

    당시 주로 Activation function으로 Sigmoid, tanh를 주로 사용하던 기존 모델과 달리 AlexNet에서는 ReLU함수를 사용

    ReLU의 비용이 다른 Activation Function보다 비교적 적은데, CIFAR-10을 가지고 비교한 결과 25% training error rate에 도달하는 시간이 6배 빠름


- Local Response Normalization

    ReLU는 Sigmoid형 함수와는 달리 input normalization이 필수는 아님. AlexNet에서는 local response normalization을 적용해 오차를 약 1~2% 줄였다고 밝힘
    
    최근에는 잘 쓰이지 않는 방법


- Overlapping Pooling

    보통 CNN같은 이미지 모델에서는 pooling unit의 크기와 stride의 크기가 같이 설정. 하지만 AlexNet에서는 Pooling unit이 더큰 Overlapping Pooling기법을 사용하여 0.4% 오차감소를 얻음.


- Data Augmentation

    “이미지-라벨”이라는 1가지 학습데이터로 이미지에 다양한 변화(사진 뒤집기, 사진 각도 전환 등등…)을 줘서 다양한 데이터셋을 생성하는 방법. 과적합 해소를 위한 방법.
    논문에서는 두가지 방법을 이용하여 변형을 적용

    1) 원본을 먼저 자른 이후 이미지를 반전 시킴

    2) PCA를 통해 RGB값을 변형함






- Dropout

    히든 레이어의 특정 뉴런의 출력을 일정 확률로 0으로 만드는것을 의미. 이는 뉴런들의 co-adaption을 줄인다. 즉, 불확실성을 부여해 특정 뉴런에 의존하지 못하도록 하는것. 이 또한 과적합 해소를 위한 방법

## Tensorflow(keras)

In [1]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential

n_classes = 1000 # class 개수
input_shape = (227, 227, 3) #이미지 인풋사이즈(227인이유: 레이어 거칠때 정수로 맞춰주려고)

model=Sequential()

############## Add Layer ##############

# Conv1
model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), padding='valid', activation='relu', input_shape=input_shape))  #
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2))) #풀링
model.add(BatchNormalization())

# Conv2
model.add(Conv2D(256, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu')) 
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))#풀링
model.add(BatchNormalization())

# Conv3
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')) 

# Conv4
model.add(Conv2D(384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))

# Conv5
model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))#풀링

# FC1
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))

# FC2
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))

# Output_layer
model.add(Dense(n_classes, activation='softmax')) #아웃풋 : 소프트맥스를 통해 1000개의 클래스 확률값 예측

#######################################

# keras summary
model.summary() # #sumary

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 27, 27, 96)       384       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                        

## Pytorch

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


class AlexNet(nn.Module):

    def __init__(self, n_classes = 1000):
        super(AlexNet, self).__init__()
        
        ############## Add Layer ##############

        # Conv1
        self.Conv_1 = nn.Sequential(
          nn.Conv2d(in_channels = 3, out_channels = 96, kernel_size = 11, stride = 4, padding = 0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 3, stride = 2),
          nn.BatchNorm2d(96))
        
        # Conv2
        self.Conv_2 = nn.Sequential(
          nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = 5, stride = 1, padding = 2),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 3, stride = 2),
          nn.BatchNorm2d(256))
        
        # Conv3
        self.Conv_3 = nn.Sequential(
          nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 3, stride = 1, padding = 1),
          nn.ReLU())
        
        # Conv4
        self.Conv_4 = nn.Sequential(
          nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 3, stride = 1, padding = 1),
          nn.ReLU())
        
        # Conv5
        self.Conv_5 = nn.Sequential(
          nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 3, stride = 2))
        
        # FC1
        self.FC1 = nn.Sequential(
          nn.Flatten(),
          nn.Dropout(0.5),
          nn.Linear(9216, 4096),
          nn.ReLU()
        )

        # FC2
        self.FC2 = nn.Sequential(
          nn.Dropout(0.5),
          nn.Linear(4096, 4096),
          nn.ReLU()
        )

        # Output_layer
        self.Output_layer = nn.Sequential(
          nn.Linear(4096, n_classes)
        )
    
        #######################################

    def forward(self, x):
        
        ############## Add Layer ##############
        out = self.Conv_1(x)
        out = self.Conv_2(out)
        out = self.Conv_3(out)
        out = self.Conv_4(out)
        out = self.Conv_5(out)
        out = self.FC1(out)
        out = self.FC2(out)
        out = self.Output_layer(out)
    
        #######################################
        
        return F.log_softmax(out)


# pytorch summary
summary(AlexNet(n_classes = 1000), (3, 227, 227)) # summary code 추가

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
       BatchNorm2d-4           [-1, 96, 27, 27]             192
            Conv2d-5          [-1, 256, 27, 27]         614,656
              ReLU-6          [-1, 256, 27, 27]               0
         MaxPool2d-7          [-1, 256, 13, 13]               0
       BatchNorm2d-8          [-1, 256, 13, 13]             512
            Conv2d-9          [-1, 384, 13, 13]         885,120
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 384, 13, 13]       1,327,488
             ReLU-12          [-1, 384, 13, 13]               0
           Conv2d-13          [-1, 256, 13, 13]         884,992
             ReLU-14          [-1, 256,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
